<a href="https://www.kaggle.com/code/masatakasuzuki/automl-h2o?scriptVersionId=120514602" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction
Welcome to my notebook!

I experiment h2o automl.
We can easily use ensamble by h2o aitoml.

Enjoy it!

1. [Import libraries](#import-libraries)
2. [Read data](#read-data)
3. [Quick review of data](#quick-review)
4. [Create and fit the model](#model)
5. [Submit your result](#submit)

<a id="import-libraries"><a/>
# Import libraries

In [1]:
!pip install h2o
from IPython.display import clear_output
clear_output()

In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import h2o
from h2o.automl import H2OAutoML

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e8/sample_submission.csv
/kaggle/input/playground-series-s3e8/train.csv
/kaggle/input/playground-series-s3e8/test.csv


<a id="read-data"></a>
# Read data

We checked directories and files. Let's read train.csv by pandas!

In [3]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e8/train.csv')
train_df.head(5)

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


<a id="quick-review"></a>
# Quick review of the data

Let's review the data quickly.

We check the number of null record. It is useful to use "info" method.

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       193573 non-null  int64  
 1   carat    193573 non-null  float64
 2   cut      193573 non-null  object 
 3   color    193573 non-null  object 
 4   clarity  193573 non-null  object 
 5   depth    193573 non-null  float64
 6   table    193573 non-null  float64
 7   x        193573 non-null  float64
 8   y        193573 non-null  float64
 9   z        193573 non-null  float64
 10  price    193573 non-null  int64  
dtypes: float64(6), int64(2), object(3)
memory usage: 16.2+ MB


There is no null record! Next, let's check the mean of each column.

The means of each columns is below. (If you want to know more about the data, See: [Feature Descriptions to help you engineer better features.](https://www.kaggle.com/competitions/playground-series-s3e8/discussion/389213), [Girdle feature and possible mistake in the documentation?](https://www.kaggle.com/competitions/playground-series-s3e8/discussion/389743))

| column name | mean |
| --- | --- |
| carat | weight of the cubic zirconia |
| cut | cut quality of the cubic zirconia. Best to Worst: Ideal > Premium > Very Good > Good > Fair |
| color | color of the cubic zirconia. Best to Worst: D > E > F > G > H > I > J . (KOSH shared helpful information. check the [link](https://www.kaggle.com/competitions/playground-series-s3e8/discussion/389589) and upvote it.|
| clarity | the absence of the inclusions and blemishes.  Best to Worst: FL > IF > VVS1 > VVS2 > VS1 > VS2 > SI1 > SI2 > I1 > I2 > I3 |
| depth | the height of the cubic zirconia |
| table | the width of the cubic zirconia |
| x | length of the cubic zirconia in mm |
| y | height of the cubic zirconia in mm |
| z | width of the cubic zirconia in mm |
| price | price of the cubic zirconia |

<a id="model"></a>
# Create and fit the model

At the first, we define preprocessing. We simple transform the label data to the number data and add some features.

In [5]:
def preprocessing(df):
    cut_labeling = {col: val for val, col in enumerate(['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'])}
    color_labeling = {col: val for val, col in enumerate(['J', 'I', 'H', 'G', 'F', 'E', 'D'])}
    clarity_labeling = {col: val for val, col in enumerate(
    ['I3', 'I2', 'I1', 'SI2', 'SI1', 'VVS2', 'VVS1', 'VS2', 'VS1', 'IF', 'FL'])}
    # to preprocess label features, map label to integer number.
    df['cut'] = df['cut'].map(cut_labeling)
    df['color'] = df['color'].map(color_labeling)
    df['clarity'] = df['clarity'].map(clarity_labeling)
    # transform x, y, z
    df['volume'] = df['x'] * df['y'] * df['z']
    df['top_surface'] = df['x'] * df['y']
    df['z_xy'] = df['z'] / (df['x'] * df['y'] + 1e-6)
    df['density'] = df['carat'] / (df['volume'] + 1e-6)
    df['table_percentage'] = (df['table'] / ((df['x'] + df['y']) / 2)) * 100
    df['depth_percentage'] = (df['depth'] / ((df['x'] + df['y']) / 2)) * 100
    df['symmetry'] = (abs(df['x'] - df['z']) + abs(df['y'] - df['z'])) / (df['x'] + df['y'] + df['z'])
    df['surface_area'] = 2 * ((df['x'] * df['y']) + (df['x'] * df['z']) + (df['y'] * df['z']))
    df['depth_to_table_ratio'] = df['depth'] / (df['table'] + 1e-6)
    df['depth_per_volume'] = df['depth'] / (df['volume'] + 1e-6)
    df['depth_per_density'] = df['depth'] / (df['density'] + 1e-6)
    df['depth_per_table'] = df['depth'] / (df['table'] + 1e-6)
    return df

In [6]:
train_df = preprocessing(train_df)
train_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,...,z_xy,density,table_percentage,depth_percentage,symmetry,surface_area,depth_to_table_ratio,depth_per_volume,depth_per_density,depth_per_table
0,0,1.52,3,4,7,62.2,58.0,7.27,7.33,4.55,...,0.085383,0.006269,794.520548,852.054795,0.287206,239.4382,1.072414,0.256531,9920.357177,1.072414
1,1,2.03,2,0,3,62.0,58.0,8.06,8.12,5.05,...,0.077161,0.006142,716.934487,766.378245,0.286387,294.3124,1.068965,0.187590,10092.700818,1.068965
2,2,0.70,4,3,8,61.2,57.0,5.69,5.73,3.50,...,0.107350,0.006134,998.248687,1071.803853,0.296247,145.1474,1.073684,0.536311,9975.106161,1.073684
3,3,0.32,4,3,8,61.6,56.0,4.38,4.41,2.71,...,0.140300,0.006113,1274.175199,1401.592719,0.293043,86.2734,1.100000,1.176789,10074.922095,1.100000
4,4,1.70,3,3,7,62.6,59.0,7.65,7.61,4.77,...,0.081936,0.006122,773.263434,820.445609,0.285572,262.0134,1.061017,0.225429,10223.955457,1.061017


The "id" column must not effect to price prediction. So we avoid "id" column.

Let's create feature data "train_x" and target data "train_y". to use keras, we convert dataframe to tensor type.

In [7]:
# feature_col = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'top_surface', 'z_xy', 'volume', 'density', 'table_percentage', 'depth_percentage', 'symmetry', 'surface_area', 'depth_to_table_ratio', 'depth_per_volume', 'depth_per_density', 'depth_per_table']
# feature_col = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'top_surface', 'z_xy', 'volume']
feature_col = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'top_surface', 'z_xy', 'volume', 'density', 'table_percentage', 'depth_percentage', 'symmetry', 'surface_area', 'depth_to_table_ratio', 'depth_per_volume', 'depth_per_density']

train_x = train_df[feature_col]
train_y = train_df['price']

In [8]:
h2o.init() 

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu220.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu220.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpiet32cyi
  JVM stdout: /tmp/tmpiet32cyi/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpiet32cyi/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,18 days
H2O_cluster_name:,H2O_from_python_unknownUser_ldv4dz
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.250 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


We need to change dataframe to h2oframe to use h2o.

In [9]:
aml = H2OAutoML(max_models = 10, seed = 1)
aml.train(x=feature_col, y='price', training_frame = h2o.H2OFrame(train_df))

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),7/10
# GBM base models (used / total),4/4
# XGBoost base models (used / total),3/3
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


check the leaderboard.

In [10]:
aml.leaderboard

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
StackedEnsemble_AllModels_1_AutoML_1_20230227_161331,574.011,329488,292.574,0.105632,329488
StackedEnsemble_BestOfFamily_1_AutoML_1_20230227_161331,575.909,331671,295.436,0.107877,331671
GBM_2_AutoML_1_20230227_161331,576.537,332395,296.19,0.108518,332395
GBM_3_AutoML_1_20230227_161331,577.323,333302,295.613,0.10756,333302
GBM_1_AutoML_1_20230227_161331,579.696,336047,293.587,0.104992,336047
GBM_4_AutoML_1_20230227_161331,582.569,339386,296.252,0.106379,339386
XGBoost_3_AutoML_1_20230227_161331,589.507,347519,306.113,0.112952,347519
XGBoost_2_AutoML_1_20230227_161331,607.135,368613,306.45,0.108867,368613
DRF_1_AutoML_1_20230227_161331,608.05,369724,310.112,0.110813,369724
XGBoost_1_AutoML_1_20230227_161331,621.345,386069,314.867,0.110386,386069


pick the best model.

In [11]:
model = aml.leader

<a id="submit"></a>
# Submit your result

submit results by using sample_sabmission.csv.

It is important to avoid index when you save submission.
Use "index_col='id'" to read_csv or "index=False" to to_csv.

In [12]:
test_df = pd.read_csv('/kaggle/input/playground-series-s3e8/test.csv')
test_df = preprocessing(test_df)
test_x = test_df[feature_col]

In [13]:
y_pred = h2o.as_list(model.predict(h2o.H2OFrame(test_x)), use_pandas=True)['predict'].tolist()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [14]:
y_pred

[873.5477108285648,
 2493.3523753570757,
 2336.547192852458,
 855.4786141402088,
 5819.78280894708,
 702.1681514902903,
 12489.330225038862,
 2958.0933148331883,
 14874.96536145462,
 1883.540958882036,
 5406.889455232336,
 829.8123750561331,
 6681.818262422652,
 1704.4379533571,
 537.5616506993906,
 1823.6425744373407,
 1032.3761259106227,
 4376.019899560056,
 8763.04918229907,
 1896.2740128111825,
 572.0714800614655,
 683.753486805662,
 8237.345471748333,
 5774.553273408748,
 3756.902877930249,
 616.8127177780037,
 2053.2567055103605,
 2834.9726809382764,
 2453.8289582891834,
 510.9132674740598,
 1354.1959782072747,
 774.430331139865,
 7000.066970346651,
 1000.654357917484,
 432.3474274800801,
 1587.896547721992,
 3525.3847262543254,
 7310.200224866791,
 12609.05098572238,
 5873.435231377286,
 651.6041932887402,
 939.8913670482344,
 1814.4220805194432,
 1764.0496718439165,
 871.159486248432,
 4493.504508153771,
 2379.9025014506888,
 13954.47203122859,
 5251.0199598155,
 699.6473739067

In [15]:
submission_df = pd.read_csv('/kaggle/input/playground-series-s3e8/sample_submission.csv', index_col='id')
submission_df['price'] = y_pred
submission_df.to_csv('submission.csv')